## Polygon meshing Workbook

After testing and devolpment code is moved to modules

In [ ]:
import numpy as np
import scipy.spatial as spatial
import matplotlib.pyplot as plt
import triangle

In [ ]:
n_r = 2
n_sides = 8
n_dim = 2
pos = np.zeros((1,n_dim))
pos_list = [pos]
for i_r in range(1,n_r+1):
    n_theta = n_sides*i_r
    r = float(i_r)/n_r
    #if i_r == n_r:
    #    n_theta = 2*n_sides*(i_r-1)
    #    theta = np.linspace(0,1,n_theta,endpoint=False)*np.pi*2.0
    #    #theta += (theta[1]-theta[0])*0.5
    #else:
    n_theta = n_sides*i_r
    theta = np.linspace(0,1,n_theta,endpoint=False)*np.pi*2.0
    pos = np.zeros((n_theta,n_dim))
    pos[:,0] = r*np.cos(theta)
    pos[:,1] = r*np.sin(theta)
    pos_list.append(pos)



In [ ]:
fig, ax = plt.subplots()
ax.set_aspect(1.0)
for pos in pos_list:
    ax.plot(pos[:,0],pos[:,1],'o')    

In [ ]:
pos_all = np.concatenate(pos_list)
print(pos_all.shape)
#plt.plot(pos_all[:,0],pos_all[:,1],'o')
tri_in = {"vertices": pos_all}    
#tri_opts = "a{:f}q{:f}iv".format(area, angle)
#tri = triangle.triangulate(tri_in, opts=tri_opts)
tri = triangle.triangulate(tri_in)
v = tri["vertices"]
fig, ax = plt.subplots()
ax.triplot(v[:,0], v[:,1], tri["triangles"])
ax.set_aspect(1.0)

In [ ]:
import tri_mesh
import importlib
importlib.reload(tri_mesh)

In [ ]:
tri_mesh.tri_calc_weights(tri)

In [ ]:
np.sum(tri["v_area"])

In [ ]:
pos = tri["vertices"]
tri_verts = tri["triangles"]
    
p1 = pos[tri_verts[:,0]]
p2 = pos[tri_verts[:,1]]
p3 = pos[tri_verts[:,2]]


In [ ]:
radius = np.sum(pos**2,axis=1)
radius.shape

In [ ]:
outer = abs(radius  - 1) < 1e-3
print(outer.shape, outer[tri_verts].shape)

In [ ]:

outer_tris = np.where(np.sum(outer[tri_verts],axis=1) == 2)[0]
outer_tris

v_i = tri_verts[outer_tris]
v = pos[tri_verts[outer_tris].T]
sA = v[1] - v[0] 
sB = v[2] - v[1]
sC = v[0] - v[2]


## Correct Area

Correct areas by adding the space between the triangle edge and the edge of the circle.

In [ ]:
n_outer_tris = outer_tris.shape[0]
def mag(x):
    return np.sum(x**2, axis=-1)**0.5
tri["area_corr"] = tri["area"]*1.0
for i in range(n_outer_tris):
    v_side = v_i[i][ np.where(outer[v_i[i]]) ]
    v_in = v_i[i][ np.where(outer[v_i[i]]== False) ]
    p_side = pos[v_side]
    r = mag(p_side)
    #print(r)
    val = np.dot(p_side[0],p_side[1])/(r[0]*r[1])
    alpha = np.arccos(val)    
    A_corr = 0.5*(alpha - np.sin(alpha))*r[0]**2
    #print(i, alpha, val, A_corr, r)

    tri["area_corr"][i] += A_corr

after correction the total read equals what is expected

In [ ]:
A = float(r.max()**2*np.pi)
print(type(A))
print("analytical corrected uncorrected")
A_corr = np.sum(tri["area_corr"])
A_tri = np.sum(tri["area"])
fmt = 3*" {:18.11e} "
print(fmt.format(A, A_corr, A_tri))
print(fmt.format(0, A_corr/A-1.0, A_tri/A-1.0 ))

In [ ]:
fig, ax = plt.subplots(2,1,sharex=True)
out = ax[0].hist(tri["area_corr"])
ax[1].hist(tri["area"], bins=out[1])